In [2]:
import os
os.getcwd()

'C:\\Users\\sergey.astakhov\\Desktop\\BmstuDeepFake'

In [3]:
import pandas as pd
import numpy as np

In [80]:
df = pd.read_json("../deep_fake_src/dfdc_dataframes/dataframe_compact_total_27_frame_10_lim100_offset0.json")

face = df.filter(like='face', axis=1).copy()

for col in face:
    name = str(col) + '_reversed'
    df[name] = face[str(col)].map(lambda x: 1.0 / max(1,x))

df = df.filter(regex='^(.(?!(var)))*$', axis=1).filter(regex='^(.(?!(noise_1)))*$', axis=1)

df.head()

,index,filename,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
0,0,aajrvbynqc.mp4,False,51984,11.603861,15.950859,22.036234,19.959687,13.405562,12.777617,...,30.0,23.0,9.0,18.0,0.392428,0.000019,0.015754,0.004667,1.0,1.0
1,1,aajrvbynqc.mp4,False,50176,15.218397,18.602677,20.166547,22.638240,15.031301,11.682786,...,9.0,10.0,8.0,20.0,0.408562,0.000020,0.015523,0.004635,1.0,1.0
2,2,aajrvbynqc.mp4,False,48400,17.841354,19.709579,22.040279,22.029932,15.643900,11.475191,...,4.0,7.0,6.0,18.0,0.425620,0.000021,0.015374,0.004661,1.0,1.0
3,3,aajrvbynqc.mp4,False,50176,14.979666,18.559879,21.796997,21.838523,15.232203,12.212530,...,3.0,11.0,9.0,14.0,0.412548,0.000020,0.015353,0.004538,1.0,1.0
4,4,aajrvbynqc.mp4,False,51076,14.595270,19.162088,21.454221,23.099652,15.149056,11.454065,...,5.0,11.0,7.0,22.0,0.403321,0.000020,0.015287,0.004425,1.0,1.0


In [81]:
df.shape

(1196, 281)

In [82]:
df_keys = pd.DataFrame(data=df[['filename', 'fake']], columns=["filename", "fake"])
df_keys_cnt = df_keys.groupby(["filename", "fake"])["filename"].count()
df_keys_cnt = pd.DataFrame(data=df_keys_cnt, columns=["filename", "cnt"])
df_keys_cnt["cnt"] = df_keys_cnt["filename"]
df_keys_cnt = df_keys_cnt.drop(columns=["filename"])
df_keys_cnt["_filename"] = df_keys_cnt.index
df_keys_cnt = df_keys_cnt.reset_index(drop=True)
df_keys_cnt["filename"] = df_keys_cnt["_filename"].map(lambda x: x[0])
df_keys_cnt["fake"] = df_keys_cnt["_filename"].map(lambda x: x[1])
# df_keys_cnt.columns = ["filename", "cnt"]
df_keys_cnt = df_keys_cnt.drop(columns=["_filename"])
df_keys_cnt.head()
# df_keys.head()

,cnt,filename,fake
0,11,aajrvbynqc.mp4,False
1,15,aasrzgbdpp.mp4,False
2,15,abplofmgwg.mp4,False
3,15,absytkidsf.mp4,True
4,15,abszlbwsaz.mp4,True


In [83]:
df_keys_cnt.shape

(89, 3)

In [84]:
df_keys_cnt = df_keys_cnt[df_keys_cnt["cnt"] > 8]
df_keys_cnt.shape

(81, 3)

In [85]:
df_keys_cnt = pd.concat([
    df_keys_cnt[df_keys_cnt.fake==True].sample(
        int(df_keys_cnt[df_keys_cnt.fake==False].shape[0]*1)), 
    df_keys_cnt[df_keys_cnt.fake==False]
])
df_keys_cnt.shape

(34, 3)

In [86]:
# from notebook2
# TODO: smart import through file

features = ['pt_24_std',
 'pt_57_hist_std',
 'pt_58_hist_noise_3',
 'pt_57_hist_noise_3',
 'pt_40_hist_noise_3',
 'pt_57_std',
 'pt_45_hist_noise_3',
 'pt_11_hist_noise_0.5',
 'pt_39_hist_noise_3',
 'pt_39_std',
 'pt_54_hist_noise_0.5',
 'overall_face_hist_noise_3',
 'pt_56_hist_noise_3',
 'pt_19_std',
 'pt_3_hist_noise_3',
 'overall_face_hist_noise_0.5',
 'overall_face_hist_std_reversed',
 'face_size_px_reversed',
 'overall_face_hist_noise_3_reversed',
 'pt_30_hist_noise_0.5',
 'overall_face_hist_noise_0.5_reversed',
 'pt_6_hist_noise_0.5',
 'overall_face_std_reversed',
 'pt_19_hist_noise_0.5',
 'pt_5_hist_noise_0.5',
 'overall_face_hist_std',
 'pt_24_hist_noise_0.5']
features

['pt_24_std',
 'pt_57_hist_std',
 'pt_58_hist_noise_3',
 'pt_57_hist_noise_3',
 'pt_40_hist_noise_3',
 'pt_57_std',
 'pt_45_hist_noise_3',
 'pt_11_hist_noise_0.5',
 'pt_39_hist_noise_3',
 'pt_39_std',
 'pt_54_hist_noise_0.5',
 'overall_face_hist_noise_3',
 'pt_56_hist_noise_3',
 'pt_19_std',
 'pt_3_hist_noise_3',
 'overall_face_hist_noise_0.5',
 'overall_face_hist_std_reversed',
 'face_size_px_reversed',
 'overall_face_hist_noise_3_reversed',
 'pt_30_hist_noise_0.5',
 'overall_face_hist_noise_0.5_reversed',
 'pt_6_hist_noise_0.5',
 'overall_face_std_reversed',
 'pt_19_hist_noise_0.5',
 'pt_5_hist_noise_0.5',
 'overall_face_hist_std',
 'pt_24_hist_noise_0.5']

In [87]:
df_merged = df_keys_cnt.merge(df, on='filename')
df_merged["fake"] = df_merged["fake_x"]
df_merged = df_merged.drop(columns=["index", "fake_x", "fake_y"])
df_merged.head()

,cnt,filename,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed,fake
0,15,aqnrvwmqbt.mp4,28224,9.749800,13.104737,6.382177,8.862213,6.903068,8.008723,6.320366,...,15.6250,14.0625,6.2500,0.729875,0.000035,0.013644,0.005341,1.0,1.0,True
1,15,aqnrvwmqbt.mp4,28224,7.520918,13.384260,11.072912,12.350819,9.345579,9.429930,4.210176,...,32.8125,6.2500,9.3750,0.736961,0.000035,0.013684,0.005508,1.0,1.0,True
2,15,aqnrvwmqbt.mp4,29584,7.016652,14.786361,10.618553,12.493504,8.497185,9.571266,4.856938,...,12.5000,10.9375,20.3125,0.692942,0.000034,0.013795,0.005053,1.0,1.0,True
3,15,aqnrvwmqbt.mp4,28224,8.211974,11.063692,5.881313,6.934332,6.113649,7.395611,4.921255,...,28.1250,20.3125,26.5625,0.733418,0.000035,0.013960,0.005407,1.0,1.0,True
4,15,aqnrvwmqbt.mp4,28224,12.846287,14.864484,9.771398,10.419415,9.480642,7.538556,3.935019,...,26.5625,9.3750,14.0625,0.722789,0.000035,0.013909,0.005391,1.0,1.0,True


In [88]:
df_merged.shape

(487, 281)

In [89]:
y = df_merged['fake']
X =  df_merged[features]
pd.DataFrame(X).head()

,pt_24_std,pt_57_hist_std,pt_58_hist_noise_3,pt_57_hist_noise_3,pt_40_hist_noise_3,pt_57_std,pt_45_hist_noise_3,pt_11_hist_noise_0.5,pt_39_hist_noise_3,pt_39_std,...,face_size_px_reversed,overall_face_hist_noise_3_reversed,pt_30_hist_noise_0.5,overall_face_hist_noise_0.5_reversed,pt_6_hist_noise_0.5,overall_face_std_reversed,pt_19_hist_noise_0.5,pt_5_hist_noise_0.5,overall_face_hist_std,pt_24_hist_noise_0.5
0,18.246140,0.935414,23.4375,18.7500,17.1875,11.862483,17.1875,0.0,14.0625,8.274585,...,0.000035,1.0,0.0,1.0,0.0,0.013644,0.0,0.0,187.228512,0.0
1,18.507804,0.800391,32.8125,25.0000,25.0000,21.720835,20.3125,0.0,12.5000,7.616270,...,0.000035,1.0,0.0,1.0,0.0,0.013684,0.0,0.0,181.556616,0.0
2,17.813370,0.785613,23.4375,20.3125,18.7500,21.132620,12.5000,0.0,10.9375,5.975679,...,0.000034,1.0,0.0,1.0,0.0,0.013795,0.0,0.0,197.920409,0.0
3,16.291264,0.785613,35.9375,20.3125,10.9375,19.743670,7.8125,0.0,14.0625,5.942083,...,0.000035,1.0,0.0,1.0,0.0,0.013960,0.0,0.0,184.941946,0.0
4,17.411207,0.684653,28.1250,28.1250,17.1875,21.408029,21.8750,0.0,10.9375,4.844834,...,0.000035,1.0,0.0,1.0,0.0,0.013909,0.0,0.0,185.491324,0.0


In [91]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X_train, X_test, y_train, y_test = train_test_split( 
    X, 
    y,
    test_size=0.20, 
    random_state=420)

In [92]:
from joblib import dump, load
clf = load('random_forest_10.pkl')

In [96]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [100]:
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures()),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=7, random_state=42))])

In [101]:
y_pred = clf.predict(X_test)
print(classification_report(
        y_test, 
        y_pred, 
        target_names=['class "real"', 'class "fakes"'], 
        zero_division=np.nan)
     )

               precision    recall  f1-score   support

 class "real"       1.00      0.93      0.96        56
class "fakes"       0.91      1.00      0.95        42

     accuracy                           0.96        98
    macro avg       0.96      0.96      0.96        98
 weighted avg       0.96      0.96      0.96        98



In [ ]:
# TODO: split by keys !!!